<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# MAT281 - Laboratorio N°09

**Objetivo**: Aplicar un flujo completo de **Machine Learning supervisado** para la clasificación de tumores mamarios, utilizando técnicas de preprocesamiento, reducción de dimensionalidad y modelos de clasificación con optimización de hiperparámetros.

> **Nota**: Puede ayudarse de algún asistente virtual como **ChatGPT, Gemini** u otros, así como del autocompletado de **Google Colab**, para avanzar en este laboratorio debido a su extensión.





<img src="https://www.svgrepo.com/show/1064/virus.svg" width = "300" align="center"/>



El **cáncer de mama** es una enfermedad caracterizada por la proliferación maligna de células epiteliales en los conductos o lobulillos mamarios. Surge cuando una célula acumula mutaciones que le otorgan la capacidad de dividirse de manera descontrolada, lo que da origen a un tumor. Este tumor puede permanecer localizado o, en casos más agresivos, invadir tejidos cercanos y propagarse a otras partes del organismo mediante metástasis.

El conjunto de datos **`BC.csv`** recopila información clínica y morfológica de pacientes con tumores mamarios, clasificados como **benignos** o **malignos**. Las características se obtienen a partir de imágenes digitalizadas de aspirados con aguja fina (FNA, por sus siglas en inglés) de masas mamarias. Dichas variables describen aspectos cuantitativos de los **núcleos celulares**, como su tamaño, forma, textura y homogeneidad.

Este tipo de información es fundamental para la detección temprana y clasificación de tumores, ya que permite entrenar modelos de **machine learning** capaces de apoyar el diagnóstico y diferenciar entre tumores benignos y malignos con mayor precisión.

A continuación, se procederá a cargar y explorar el conjunto de datos:



In [2]:
# Importar librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importar herramientas de Scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configuración de gráficos
%matplotlib inline
sns.set_palette("deep", desat=0.6)
sns.set(rc={'figure.figsize': (11.7, 8.27)})

# Cargar y preparar los datos
df = pd.read_csv("https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/BC.csv")
df.set_index('id', inplace=True)

# Transformación de la variable objetivo
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0}).astype(int)

# Visualizar las primeras filas del DataFrame
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678



Con base en la información presentada, resuelva las siguientes tareas. Asegúrese de:

* Incluir el **código necesario** para ejecutar cada análisis.
* Explicar de manera **clara y fundamentada** los resultados obtenidos.
* Describir el **proceso seguido**, justificando las decisiones tomadas en cada etapa (preprocesamiento, elección de técnicas y parámetros, interpretación de resultados).





1. **Análisis exploratorio profundo (EDA):**

   * Examine la distribución de las variables, identifique valores atípicos y analice la correlación entre características.
   * Visualice las diferencias más relevantes entre tumores **benignos** y **malignos** utilizando gráficos adecuados (boxplots, histogramas, mapas de calor).
   * Discuta qué variables parecen tener mayor capacidad discriminativa.


In [3]:
print('Tamaño')
print(df.shape)
print('--------------------------------------------------------------------')
print('Columnas')
print(df.columns)
print('--------------------------------------------------------------------')
print('Tipos de datos')
print(df.info())
print('--------------------------------------------------------------------')
print('Estadistica')
print(df.describe())
print('--------------------------------------------------------------------')
print('Valores unicos')
print(df.nunique())
print('--------------------------------------------------------------------')
print('Valores nulos')
print(df.isna().sum())

Tamaño
(569, 31)
--------------------------------------------------------------------
Columnas
Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')
--------------------------------------------------------------------
Tipos de datos
<class 'pandas.core.frame.DataFrame'>
Index: 569 entries, 842302 to 92751
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------     


2. **Preprocesamiento de datos:**

   * Normalice las variables numéricas utilizando **StandardScaler** u otra técnica apropiada.
   * Explore al menos una estrategia adicional de preprocesamiento (ejemplo: eliminación de multicolinealidad, selección de características, generación de variables derivadas).
   * Justifique sus elecciones.


In [4]:

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.drop('diagnosis', axis=1))
df_scaled = pd.DataFrame(df_scaled, columns=df.columns[1:], index=df.index)
df_scaled=pd.concat([df['diagnosis'],df_scaled],axis=1).reset_index(drop=True)
df_scaled.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,2.217515,...,1.886690,-1.359293,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015
1,1,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,0.001392,...,1.805927,-0.369203,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190
2,1,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,0.939685,...,1.511870,-0.023974,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391
3,1,-0.768909,0.253732,-0.592687,-0.764464,3.283553,3.402909,1.915897,1.451707,2.867383,...,-0.281464,0.133984,-0.249939,-0.550021,3.394275,3.893397,1.989588,2.175786,6.046041,4.935010
4,1,1.750297,-1.151816,1.776573,1.826229,0.280372,0.539340,1.371011,1.428493,-0.009560,...,1.298575,-1.466770,1.338539,1.220724,0.220556,-0.313395,0.613179,0.729259,-0.868353,-0.397100



3. **Reducción de dimensionalidad:**

   * Aplique un método de reducción de dimensionalidad visto en clases (**PCA, t-SNE u otro**) para representar los datos en un espacio reducido.
   * Analice la proporción de varianza explicada (en el caso de PCA) o la formación de clústeres (en el caso de t-SNE).
   * Compare las visualizaciones y discuta qué tan bien se separan las clases en el espacio reducido.


In [6]:

pc=PCA()
pc.fit(df_scaled)
explained_variance_ratio=pc.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)
n_components = np.argmax(cumulative_variance >= 0.95) + 1  #umbral del 95% de varianza
print(f"Número óptimo de componentes: {n_components}")



X=df_scaled.drop('diagnosis',axis=1)
y=df_scaled['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pca=PCA(n_components=11)
X_train_pca = pca.fit_transform(X_train)
X_eval_pca = pca.transform(X_test)

Número óptimo de componentes: 11



4. **Modelado y evaluación:**

   * Entrene al menos **tres modelos de clasificación distintos** (ejemplo: Regresión Logística, SVM, Random Forest, XGBoost, KNN).
   * Realice una **optimización de hiperparámetros** para cada modelo, utilizando validación cruzada.
   * Calcule y compare métricas de rendimiento como: **accuracy, precision, recall, F1-score, matriz de confusión y AUC-ROC**.
   * Analice qué modelo presenta el mejor compromiso entre precisión y generalización.


In [10]:
# Logistic Regression
# Optimizacion de hiperparametros
param_grid1={
    'C':[0.001,0.01,0.1,1,10,100],
    'penalty':['l1','l2'],
    'solver':['liblinear','saga'],
    'max_iter':[100,200,300]
}
model1=LogisticRegression()
grid_search1=GridSearchCV(
    estimator=model1,
    param_grid=param_grid1,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)
grid_search1.fit(X_train_pca,y_train)
print("Mejor precisión en validación cruzada:", grid_search1.best_score_)
print("Mejores parámetros:", grid_search1.best_params_)
mejor_modelo1 = grid_search1.best_estimator_
mejor_modelo1.fit(X_train_pca, y_train)
print("Precisión en el conjunto de evaluación: {:.3f}".format(mejor_modelo1.score(X_eval_pca, y_test)))
y_pred1=mejor_modelo1.predict(X_eval_pca)







Fitting 5 folds for each of 72 candidates, totalling 360 fits
Mejor precisión en validación cruzada: 0.9780219780219781
Mejores parámetros: {'C': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Precisión en el conjunto de evaluación: 0.991


Notemos que la precision obtenida luego de escoger el mejor modelo con Grid es mayor que la esperada.

In [11]:
# SVC
# Optimizacion de hiperparametros
param_grid2={
    'C':[0.1,1,10,100],
    'kernel':['linear','rbf','poly'],
    'gamma':['scale','auto',0.1,1,10],
    'degree':[3,4]
}
model2=SVC()
grid_search2=GridSearchCV(
    estimator=model2,
    param_grid=param_grid2,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)
grid_search2.fit(X_train_pca,y_train)
print("Mejor precisión en validación cruzada:", grid_search2.best_score_)
print("Mejores parámetros:", grid_search2.best_params_)
mejor_modelo2 = grid_search2.best_estimator_
mejor_modelo2.fit(X_train_pca, y_train)
print("Precisión en el conjunto de evaluación: {:.3f}".format(mejor_modelo2.score(X_eval_pca, y_test)))
y_pred2=mejor_modelo2.predict(X_eval_pca)


Fitting 5 folds for each of 120 candidates, totalling 600 fits
Mejor precisión en validación cruzada: 0.9780219780219781
Mejores parámetros: {'C': 0.1, 'degree': 3, 'gamma': 'scale', 'kernel': 'linear'}
Precisión en el conjunto de evaluación: 0.991


In [ ]:
#Random Forest
# Optimizacion de parametros
param_grid3={
    'n_estimators':[10,50,100,200],
    'max_depth':[10,20,30],
    'min_samples_split':[2,5,10],
    'min_samples_leaf':[1,2,4],
    'max_features':['sqrt','log2']
}
model3=RandomForestClassifier()
grid_search3=GridSearchCV(
    estimator=model3,
    param_grid=param_grid3,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)
grid_search3.fit(X_train_pca,y_train)
print("Mejor precisión en validación cruzada:", grid_search3.best_score_)
print("Mejores parámetros:", grid_search3.best_params_)
mejor_modelo3=grid_search3.best_estimator_
mejor_modelo3.fit(X_train_pca,y_train)
print("Precisión en el conjunto de evaluación: {:.3f}".format(mejor_modelo3.score(X_eval_pca, y_test)))
y_pred3=mejor_modelo3.predict(X_eval_pca)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits



5. **Conclusiones y reflexiones:**

   * Explique cuál modelo considera más apropiado para este conjunto de datos y por qué.
   * Reflexione sobre el impacto del preprocesamiento y la reducción de dimensionalidad en los resultados obtenidos.
   * Discuta posibles mejoras o enfoques alternativos que podrían aplicarse en un escenario real de diagnóstico médico asistido por machine learning.



Si bien la precesion en los dos primeros modelos 1, lo que indica un overfitting, la exactitud es mas fiable, por la forma en que se calcula. Segun esta los dos primeros modelos predominan. Vemos la importancia de estudiar los hiperparametros para conseguir un mejor ajuste en los datos. Aun que sea del orden de 3 decimales, siempre es una mejora.